In [ ]:
import pandas as pd
from cartopy import crs
from bokeh.models import HoverTool
import geoviews as gv
import geoviews.feature as gf

gv.extension("bokeh")

## Load INTERMAGNET & SuperMAG locations

Sources:
 - https://www.intermagnet.org/imos/imotblobs-eng.php
 - https://supermag.jhuapl.edu/inventory
 
Not including other WDC locations:
 - http://www.wdc.bgs.ac.uk/data.html

In [ ]:
df_iaga = pd.read_csv("data/IAGA_observatories.txt", sep="\t", header=1)
# Remove spurious spaces in column names
for col in df_iaga.columns:
    df_iaga = df_iaga.rename(columns={col:col.strip()})
# Strip spurious spaces in each column
for col in df_iaga.columns:
    df_iaga[col] = df_iaga[col].str.strip()
# Identify retired observatories indicated by *
for i, row in df_iaga.iterrows():
    df_iaga.loc[i, "Former observatory"] = "*" in row["IAGA"]
    df_iaga.loc[i, "IAGA"] = row["IAGA"].replace("*", "")
df_iaga

In [ ]:
df_supermag = pd.read_csv("data/SuperMAG-Inventory-60s-2021-04-17.txt", sep="\t", header=1)
df_supermag

In [ ]:
# IAGA observatories not in SuperMAG list
iaga_codes = set(df_iaga["IAGA"].values)
codes_supermag = set(df_supermag["IAGA"].values)
{i for i in iaga_codes if i not in codes_supermag}

## Construct dataset to visualise

In [ ]:
df = df_supermag[["IAGA", "GLON", "GLAT", "Station name"]]
df = df.rename(columns={"IAGA":"IAGA_code", "Station name":"Station_name", "GLON":"Longitude", "GLAT":"Latitude"})
df["Catalog"] = "SuperMAG"
for i, row in df.iterrows():
    if row["IAGA_code"] in iaga_codes:
        df.loc[i, "Catalog"] = "+ INTERMAGNET"
df

## Construct visualisation

Make something like: https://intermagnet.github.io/metadata/map

Check guides at https://geoviews.org/

In [ ]:
points = gv.Points(df, kdims=["Longitude", "Latitude"], vdims=["IAGA_code", "Station_name", "Catalog"])

In [ ]:
features = gv.Overlay([gf.ocean, gf.land, gf.borders])

In [ ]:
# Manually change the hover to remove Lat/lon which is incorrect
# Shouldn't this be fixed already? ref https://github.com/holoviz/geoviews/issues/100
hover = HoverTool(tooltips=[("IAGA code", "@{IAGA_code}"), ("Station name", "@{Station_name}")])

In [ ]:
options = dict(
    alpha=0.6, size=7, color="Catalog", cmap="Spectral",
    tools=[hover],
    global_extent=True, width=800, height=400,
    projection=crs.Mollweide()
)

In [ ]:
fig = features * points.opts(**options)
fig

In [ ]:
!mkdir outputs -p
gv.save(fig, "outputs/hvplot_mag_obs_map.html")